<a href="https://colab.research.google.com/github/MannyD95/DigitalTwinMVP/blob/main/Digital_Twin_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install pycaret pandas numpy matplotlib scikit-learn

In [ ]:
import os
import pandas as pd
import re  # Regex for parsing irregular whitespace

# Define base directory
base_path = "/content/drive/MyDrive/motion-and-heart-rate-from-a-wrist-worn-wearable-and-labeled-sleep-from-polysomnography-1.0.0"
folders = ["heart_rate", "motion", "labels", "steps"]

# Dictionary to hold DataFrames
data_dict = {}

# Loop through each folder
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        continue

    all_files = os.listdir(folder_path)
    all_data = []

    print(f"\nProcessing folder: {folder}")

    for file in all_files:
        file_path = os.path.join(folder_path, file)
        try:
            # Skip invalid or empty files
            if os.path.getsize(file_path) == 0:  # File size check for emptiness
                print(f"Skipping empty file: {file}")
                continue

            # Check the first line of the file to avoid HTML or empty files
            with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                first_line = f.readline()
                if not first_line.strip() or '<html>' in first_line:
                    print(f"Skipping invalid file: {file}")
                    continue

            # Handle motion files, which may have irregular whitespace
            if folder == "motion":
                print(f"Processing motion file: {file_path}")
                data = []
                with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                    for line in f:
                        split_line = re.split(r'\s+', line.strip())
                        if len(split_line) == 4:  # Ensure valid row with 4 columns
                            data.append(split_line)

                # Convert to DataFrame if we have valid data
                if data:
                    df = pd.DataFrame(data, columns=["timestamp", "x_acc", "y_acc", "z_acc"])
                    # Convert columns to appropriate types
                    df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")
                    df["x_acc"] = pd.to_numeric(df["x_acc"], errors="coerce")
                    df["y_acc"] = pd.to_numeric(df["y_acc"], errors="coerce")
                    df["z_acc"] = pd.to_numeric(df["z_acc"], errors="coerce")
                    df = df.dropna()  # Drop rows with NaN after conversion
                    all_data.append(df)
                    print(f"Loaded motion file: {file} with {len(df)} rows.")
                else:
                    print(f"No valid data found in motion file: {file}")
            else:
                # Handle heart_rate, labels, and steps normally
                if folder == "heart_rate":
                    df = pd.read_csv(file_path, sep=",", header=None, names=["timestamp", "heart_rate"],
                                     engine="python", on_bad_lines="skip")
                elif folder == "labels":
                    df = pd.read_csv(file_path, sep="\s+", header=None, names=["timestamp", "sleep_stage"],
                                     engine="python", on_bad_lines="skip")
                elif folder == "steps":
                    df = pd.read_csv(file_path, sep="\s+", header=None, names=["timestamp", "steps"],
                                     engine="python", on_bad_lines="skip")
                else:
                    df = pd.DataFrame()

                if not df.empty:
                    all_data.append(df)
                    print(f"Loaded file: {file} with {len(df)} rows.")

        except Exception as e:
            print(f"Error reading {file}: {e}")

    # Combine valid files into a single DataFrame for the folder
    if all_data:
        data_dict[folder] = pd.concat(all_data, ignore_index=True)
        print(f"Loaded data from {folder}: {len(all_data)} files with {len(data_dict[folder])} rows total.")
    else:
        print(f"No valid data found in folder: {folder}")

# Display summary of loaded data
for folder in folders:
    if folder in data_dict:
        print(f"\nSummary of {folder} Data:")
        print(data_dict[folder].info())
        print(data_dict[folder].head())
    else:
        print(f"\nNo data loaded for {folder}.")

In [ ]:
print("Heart Rate Data:")
print(data_dict['heart_rate'].head())

print("Motion Data:")
print(data_dict['motion'].head())

print("Labels Data:")
print(data_dict['labels'].head())

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import multiprocessing as mp

# Convert numeric columns to more memory-efficient types
def optimize_dtypes(df):
    for col in df.select_dtypes(include=['float64', 'float32']).columns:
        df[col] = df[col].astype(np.float32)
    for col in df.select_dtypes(include=['int64', 'int32', 'int']).columns:
        df[col] = df[col].astype(np.int32)
    return df

# Fix the timestamp in steps to replace commas with periods, then convert to float
data_dict['steps']['timestamp'] = data_dict['steps']['timestamp'].astype(str).str.replace(',', '.', regex=False)

# Convert all timestamps to float and optimize dtypes
data_dict['steps']['timestamp'] = data_dict['steps']['timestamp'].astype(np.float32)
data_dict['heart_rate']['timestamp'] = data_dict['heart_rate']['timestamp'].astype(np.float32)
data_dict['motion']['timestamp'] = data_dict['motion']['timestamp'].astype(np.float32)
data_dict['labels']['timestamp'] = data_dict['labels']['timestamp'].astype(np.float32)

data_dict['heart_rate'] = optimize_dtypes(data_dict['heart_rate'])
data_dict['motion'] = optimize_dtypes(data_dict['motion'])
data_dict['labels'] = optimize_dtypes(data_dict['labels'])
data_dict['steps'] = optimize_dtypes(data_dict['steps'])

# Sort DataFrames by timestamp before merge_asof
data_dict['heart_rate'].sort_values('timestamp', inplace=True)
data_dict['motion'].sort_values('timestamp', inplace=True)
data_dict['labels'].sort_values('timestamp', inplace=True)

# Function to process a chunk of motion data
def process_chunk(chunk):
    chunk = chunk.sort_values('timestamp')

    # Merge motion and heart_rate for this chunk
    merged = pd.merge_asof(
        chunk,
        data_dict['heart_rate'],
        on='timestamp',
        direction='nearest'
    )

    # Merge labels data
    merged = pd.merge_asof(
        merged.sort_values('timestamp'),
        data_dict['labels'],
        on='timestamp',
        direction='nearest'
    )

    return merged

# Split motion data into chunks to reduce memory usage
num_chunks = 10  # Adjust based on memory constraints
motion_chunks = np.array_split(data_dict['motion'], num_chunks)

# Use parallel processing to merge chunks
with mp.Pool(processes=4) as pool:  # Adjust processes based on CPU cores
    results = list(tqdm(pool.imap(process_chunk, motion_chunks), total=num_chunks, desc='Merging Chunks'))

# Concatenate all chunks into one DataFrame
merged_data = pd.concat(results, ignore_index=True)

# Remove rows with invalid sleep stages (-1)
merged_data = merged_data[merged_data['sleep_stage'] != -1]

# Map sleep stages to a "Fitbit scale"
sleep_stage_mapping = {0: 20, 1: 40, 2: 60, 3: 80, 5: 100}
merged_data['sleep_quality'] = merged_data['sleep_stage'].map(sleep_stage_mapping)

# Drop unused columns
merged_data.drop(['sleep_stage'], axis=1, inplace=True)

print("Merged Data with Sleep Quality:")
print(merged_data.head())
print("Shape of Merged Data:", merged_data.shape)

# Sort by timestamp to create lag features
merged_data.sort_values(by='timestamp', inplace=True)

# Add lag features for previous metrics (1-step lag)
merged_data['prev_heart_rate'] = merged_data['heart_rate'].shift(1)
merged_data['prev_x_acc'] = merged_data['x_acc'].shift(1)
merged_data['prev_y_acc'] = merged_data['y_acc'].shift(1)
merged_data['prev_z_acc'] = merged_data['z_acc'].shift(1)

# Drop rows with NaN values introduced by lagging
final_data = merged_data.dropna()

print("Final Data for Modeling with Lag Features:")
print(final_data.head())
print("Shape of Final Data:", final_data.shape)


In [ ]:
!pip install pycaret dask[complete] tqdm

import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from pycaret.regression import setup, compare_models

# Ensure `final_data` is already in the environment
if 'final_data' not in globals():
    raise ValueError("`final_data` is not defined. Please define final_data before running this code.")

########################################
# Step 1: Save final_data to Parquet
########################################
parquet_path = "final_data.parquet"
final_data.to_parquet(parquet_path, compression='snappy')
del final_data  # Free memory

########################################
# Step 2: Load with Dask in a memory-efficient manner
########################################
ddf = dd.read_parquet(parquet_path, engine='pyarrow')

########################################
# Step 3: Sampling the data
########################################
sample_frac = 0.0001  # Adjust if needed

with ProgressBar():
    ddf_sample = ddf.sample(frac=sample_frac, random_state=42)
    sample_data = ddf_sample.compute()

print("Shape of Sampled Data:", sample_data.shape)

# Optimize memory for sample_data by converting columns to smaller dtypes
for col in sample_data.select_dtypes(include=['float64']).columns:
    sample_data[col] = sample_data[col].astype(np.float32)
for col in sample_data.select_dtypes(include=['int64']).columns:
    sample_data[col] = sample_data[col].astype(np.int32)

########################################
# Step 4: Run PyCaret on the sampled data
########################################
target_col = 'sleep_quality'
if target_col not in sample_data.columns:
    raise ValueError(f"Target column '{target_col}' not found in sampled data.")

# Setup PyCaret regression without 'interactive' parameter
exp = setup(
    data=sample_data,
    target=target_col,
    session_id=123,
    train_size=0.8,
    html=False,     # Suppress HTML output
    verbose=False   # Reduce printed logs
)

print("Comparing models...")
best_model = compare_models()  # Compare models on the sampled data

print("Best model found:", best_model)


In [ ]:
from pycaret.regression import save_model

# Assuming 'best_model' is your trained model
save_model(best_model, model_name='my_best_model')


In [ ]:
from pycaret.regression import save_model
save_model(best_model, model_name='/content/drive/MyDrive/motion-and-heart-rate-from-a-wrist-worn-wearable-and-labeled-sleep-from-polysomnography-1.0.0/my_best_model')


In [ ]:
!pip install dash jupyter-dash


In [ ]:
!pip install pycaret


In [ ]:
from pycaret.regression import load_model

# Load the model
model = load_model('/content/drive/MyDrive/motion-and-heart-rate-from-a-wrist-worn-wearable-and-labeled-sleep-from-polysomnography-1.0.0/my_best_model')


Transformation Pipeline and Model Successfully Loaded


In [ ]:
!pip install gradio


In [ ]:
!pip install ipdb

In [ ]:
import gradio as gr
import pandas as pd
import traceback
from pycaret.regression import load_model
import random

# 1. Load your trained KNN model
model_path = "/content/drive/MyDrive/motion-and-heart-rate-from-a-wrist-worn-wearable-and-labeled-sleep-from-polysomnography-1.0.0/my_best_model"
model = load_model(model_path)

def activity_to_acceleration(level):
    mapping = {
        "Low": 0.3 + (random.random() * 0.4),      # 0.3-0.7
        "Moderate": 1.0 + (random.random() * 0.8),  # 1.0-1.8
        "High": 1.8 + (random.random() * 1.2)       # 1.8-3.0
    }
    return mapping.get(level, 1.0)

def calculate_health_score(heart_rate, steps, activity_level):
    # Define expected ranges based on activity level
    activity_ranges = {
        "Low": {
            "hr_optimal_range": (60, 75),
            "steps_target": 8000,
            "steps_min": 4000,
        },
        "Moderate": {
            "hr_optimal_range": (55, 70),
            "steps_target": 12000,
            "steps_min": 7000,
        },
        "High": {
            "hr_optimal_range": (45, 65),
            "steps_target": 15000,
            "steps_min": 10000,
        }
    }

    current_range = activity_ranges[activity_level]
    hr_min, hr_max = current_range["hr_optimal_range"]

    # Heart rate scoring - more punitive for extreme values
    if hr_min <= heart_rate <= hr_max:
        hr_score = 1.0
    elif heart_rate < hr_min:
        # Penalize very low heart rates more severely
        hr_score = max(0.3, 0.7 + (heart_rate - (hr_min - 10)) / (hr_min - (hr_min - 10)) * 0.3)
    else:
        # Penalize high heart rates more severely
        hr_score = max(0.2, 1.0 - ((heart_rate - hr_max) / 30))

    # Steps scoring with minimum threshold
    steps_min = current_range["steps_min"]
    steps_target = current_range["steps_target"]

    if steps < steps_min:
        # Severe penalty for very low steps
        steps_score = max(0.2, steps / steps_min * 0.5)
    else:
        steps_score = min(1.0, (steps - steps_min) / (steps_target - steps_min))

    # Activity level consistency score - more punitive for inconsistencies
    consistency_score = 1.0

    # Severe inconsistency penalties
    if activity_level == "High":
        if steps < 8000:
            consistency_score *= 0.4  # Severe penalty for high activity but very low steps
        elif steps < 10000:
            consistency_score *= 0.6
        if heart_rate > 80:
            consistency_score *= 0.5  # High resting HR inconsistent with high activity

    elif activity_level == "Moderate":
        if steps < 5000:
            consistency_score *= 0.5
        if heart_rate > 90:
            consistency_score *= 0.6

    elif activity_level == "Low":
        if steps > 15000:
            consistency_score *= 0.6  # Inconsistent with claimed low activity
        if heart_rate < 50:
            consistency_score *= 0.7  # Too low HR for low activity

    # Extreme cases penalties
    if heart_rate > 100:
        consistency_score *= 0.4  # Severe penalty for very high resting HR
    if steps < 2000:
        consistency_score *= 0.3  # Severe penalty for extremely sedentary

    # Calculate base score with adjusted weights
    base_score = (hr_score * 0.35 + steps_score * 0.35 + consistency_score * 0.3) * 100

    # Additional penalties for extreme poor scenarios
    if steps < 2000 and heart_rate > 90:
        base_score *= 0.6  # Severe penalty for very unhealthy combination

    # Cap the minimum score at 20 to allow for very poor scenarios
    base_score = max(20, base_score)

    print(f"\nDetailed Score Breakdown:")
    print(f"Heart Rate Score: {hr_score:.2f}")
    print(f"Steps Score: {steps_score:.2f}")
    print(f"Consistency Score: {consistency_score:.2f}")
    print(f"Final Base Score: {base_score:.2f}")

    return base_score, hr_score, steps_score, consistency_score

def predict_and_suggest(activity_level, heart_rate, steps):
    try:
        acceleration = activity_to_acceleration(activity_level)

        # Calculate comprehensive health score
        base_score, hr_score, steps_score, consistency_score = calculate_health_score(heart_rate, steps, activity_level)

        # Debug prints
        print(f"\nDetailed Metrics Analysis:")
        print(f"Activity Level: {activity_level}")
        print(f"Heart Rate: {heart_rate} bpm (Score: {hr_score:.2f})")
        print(f"Steps: {steps:,} (Score: {steps_score:.2f})")
        print(f"Consistency Score: {consistency_score:.2f}")
        print(f"Final Score: {base_score:.2f}")

        suggestions = []
        if base_score >= 85:
            category = f"Excellent: {base_score:.2f} 🟢"
            suggestions = [
                "🌟 Excellent metrics with great consistency!",
                f"\nMetrics Analysis:",
                f"• Heart Rate ({heart_rate} bpm): Optimal for your activity level",
                f"• Steps ({steps:,}): Excellent activity volume",
                f"• Activity Level ({activity_level}): Well-aligned with your metrics",
                "\nRecommendations:",
                "• Maintain your current balanced routine",
                "• Consider heart rate variability tracking for further optimization",
                "• Focus on recovery quality to support your activity level"
            ]
        elif 70 <= base_score < 85:
            category = f"Good: {base_score:.2f} 🟢"
            suggestions = [
                "🎯 Strong metrics with room for fine-tuning:",
                f"\nMetrics Analysis:"
            ]

            if consistency_score < 0.9:
                suggestions.extend([
                    "• Your metrics show slight misalignment:",
                    f"  - Activity level ({activity_level}) vs. Steps ({steps:,})",
                    f"  - Consider if your activity level assessment is accurate"
                ])

            if hr_score < 0.9:
                suggestions.extend([
                    f"• Heart Rate ({heart_rate} bpm):",
                    "  - Consider timing of exercise relative to sleep",
                    "  - Monitor caffeine intake timing"
                ])

        elif 55 <= base_score < 70:
            category = f"Fair: {base_score:.2f} 🟡"
            suggestions = [
                "💪 Good foundation with opportunities for improvement:",
                f"\nMetrics Analysis:"
            ]

            if consistency_score < 0.8:
                suggestions.extend([
                    "• Metric Alignment Needs Work:",
                    f"  - Your {activity_level} activity level doesn't match your:",
                    f"    * Daily steps ({steps:,})",
                    f"    * Resting heart rate ({heart_rate} bpm)",
                    "  - Consider reassessing your typical activity level"
                ])

            if hr_score < 0.8:
                suggestions.extend([
                    "• Heart Rate Optimization:",
                    "  - Implement regular relaxation practices",
                    "  - Review exercise timing and intensity",
                    "  - Monitor stress levels and management"
                ])

        else:
            category = f"Poor: {base_score:.2f} 🔴"
            suggestions = [
                "⚠️ Your metrics indicate significant room for improvement:",
                f"\nCurrent Status (Needs Attention):",
                f"• Activity Level: {activity_level}",
                f"• Resting Heart Rate: {heart_rate} bpm {'(Concerning - too high)' if heart_rate > 90 else '(Needs improvement)'}",
                f"• Daily Steps: {steps:,} {'(Severely low)' if steps < 3000 else '(Below recommended minimum)'}",
                "\nPriority Actions:",
                "1. Immediate Health Focus:",
                "   • Consult with healthcare provider about heart rate" if heart_rate > 90 else "   • Begin with simple walking routine",
                "   • Start with 5-minute walking breaks every hour",
                "   • Focus on stress reduction techniques",
                "\n2. Progressive Building:",
                f"   • Set initial goal: {max(2000, steps)} → {max(4000, steps + 2000)} steps",
                "   • Add 500 steps per week",
                "   • Consider gentle morning stretches",
                "\n3. Lifestyle Adjustments:",
                "   • Reduce sitting time with hourly breaks",
                "   • Improve sleep environment and routine",
                "   • Monitor and log daily activity"
            ]

        return category, "\n".join(suggestions)

    except Exception as e:
        error_trace = traceback.format_exc()
        print(f"Error: {e}")
        print("Traceback:")
        print(error_trace)
        return f"Error: {str(e)}", "No suggestions available due to error."

# Create Gradio Interface
iface = gr.Interface(
    fn=predict_and_suggest,
    inputs=[
        gr.Dropdown(choices=["Low", "Moderate", "High"], value="Moderate", label="Activity Level"),
        gr.Slider(40, 120, step=1, value=75, label="Average Heart Rate (bpm)"),
        gr.Slider(0, 30000, step=500, value=10000, label="Total Steps in a Day")
    ],
    outputs=[
        gr.Textbox(label="Predicted Sleep Quality"),
        gr.Textbox(label="Suggestions to Improve")
    ],
    title="Sleep Quality Predictor",
    description="Get insights into your sleep quality based on your activity level, heart rate, and daily steps."
)

# Launch the app
iface.launch()

Transformation Pipeline and Model Successfully Loaded
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6c176220ddee7ed155.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
